In [3]:
from keras import Input, layers

# tensor
input_tensor = Input(shape=(32,))

# layer object is function
dense = layers.Dense(32, activation='relu')

# layer object called by tensor, retuns tensor
output_tensor = dense(input_tensor)